In [ ]:
import os
import sys
import matplotlib.pyplot as plt
import numpy as np
from pathlib import Path
import h5py
import requests

# ======================================================================================
# Download the darkmode.mplstyle stylesheet and use it
# ======================================================================================
# Download the darkmode.mplstyle stylesheet from the website repository
url = (
    r"https://raw.githubusercontent.com/vincentvdschaft/quartz-website/v4/"
    r"figure-generation/darkmode.mplstyle")
r = requests.get(url)
# Write the downloaded stylesheet to a file
with open('stylesheet.mplstyle', 'wb') as f:
    f.write(r.content)
# Use the stylesheet
plt.style.use('stylesheet.mplstyle')

# Define functions to compute the CNR and the gCNR

In [ ]:
def compute_cnr(pixels_a, pixels_b):
    """Computes the Contrast-to-Noise Ratio (CNR) between two sets of pixel intensities.
    """
    mu_a = np.mean(pixels_a)
    mu_b = np.mean(pixels_b)

    sigma_a = np.std(pixels_a)
    sigma_b = np.std(pixels_b)

    cnr = np.abs(mu_a-mu_b)/np.sqrt(sigma_a**2+sigma_b**2)
    return cnr


def compute_gcnr(region_a: np.ndarray, region_b: np.ndarray, bins: int = 256):
    """Computes the Generalized Contrast-to-Noise Ratio (gCNR) between two sets of pixel
    intensities. The GCNR is computed based on the histogram of the pixel intensities
    with the specified number of bins.

    Parameters
    ----------
    region_a : np.ndarray
        The first set of pixel intensities.
    region_b : np.ndarray
        The second set of pixel intensities.
    bins : int
        The number of bins to use for the histogram.

    Returns
    -------
    float
        The GCNR value.
    """

    # Flatten arrays of pixels
    region_a = region_a.flatten()
    region_b = region_b.flatten()

    # Compute a histogram for the two regions together to find a good set of shared bins
    _, bins = np.histogram(np.concatenate((region_a, region_b)), bins=bins)

    # Compute the histograms for the two regions individually with the shared bins
    hist_region_1, _ = np.histogram(region_a, bins=bins, density=True)
    hist_region_2, _ = np.histogram(region_b, bins=bins, density=True)

    # Normalize the histograms to unit area
    hist_region_1 /= hist_region_1.sum()
    hist_region_2 /= hist_region_2.sum()

    # Compute and return the GCNR
    return 1 - np.sum(np.minimum(hist_region_1, hist_region_2))

# Create an image with two regions

In [ ]:
# ======================================================================================
# Define the mean and standard deviation of the pixel intensities in region A and B
# ======================================================================================
mu_a = 0.25
sigma_a = 0.15

mu_b = 0.75
sigma_b = 0.15

# Generate random pixel intensities for region A and B
pixels_region_a = np.random.randn(100, 100)*sigma_a+mu_a
pixels_region_b = np.random.randn(100, 100)*sigma_b+mu_b

# Clip the pixel values to the range [0, 1]
pixels_region_a = np.clip(pixels_region_a, 0, 1)
pixels_region_b = np.clip(pixels_region_b, 0, 1)

# Concatenate the two regions into a single image
image = np.concatenate([pixels_region_a, pixels_region_b], axis=1)

# ======================================================================================
# Define probability density functions for the pixel intensities in region A and B
# ======================================================================================
def p_a(x):
    """Computes the gaussian probability density function for the region A."""
    return 1/(sigma_a*np.sqrt(2*np.pi))*np.exp(-0.5*((x-mu_a)/sigma_a)**2)

def p_b(x):
    """Computes the gaussian probability density function for the region B."""
    return 1/(sigma_b*np.sqrt(2*np.pi))*np.exp(-0.5*((x-mu_b)/sigma_b)**2)

In [ ]:
# Define an s-curve function
def s_curve(x):
    return 1/(1+np.exp(-10*(x-0.5)))

# ======================================================================================
# Compute the CNR and GCNR for the original and stretched pixel intensities
# ======================================================================================
cnr = compute_cnr(pixels_region_a, pixels_region_b)
cnr_stretched = compute_cnr(s_curve(pixels_region_a), s_curve(pixels_region_b))

gcnr = compute_gcnr(pixels_region_a, pixels_region_b)
gcnr_stretched = compute_gcnr(s_curve(pixels_region_a), s_curve(pixels_region_b))

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(7, 3.5))
ax_image = axes[0]
ax_image_scurve = axes[2]
ax_s_curve = axes[1]

# ======================================================================================
# Plot the original image
# ======================================================================================
ax_image.imshow(image, cmap='gray', vmin=0, vmax=1)
ax_image.axis('off')
ax_image.set_title(f"Image\nCNR: {cnr:.2f}\nGCNR: {gcnr:.2f}")
# Add text A and B to the image
ax_image.text(50, 50, "A", color="white", fontsize=20, ha="center", va="center")
ax_image.text(150, 50, "B", color="black", fontsize=20, ha="center", va="center")

# ======================================================================================
# Plot the S-curve transformed image
# ======================================================================================
ax_image_scurve.imshow(s_curve(image), cmap='gray', vmin=0, vmax=1)
ax_image_scurve.axis("off")
ax_image_scurve.set_title(
    f"Image after applying s-curve\n"
    f"CNR: {cnr_stretched:.2f}\nGCNR: {gcnr_stretched:.2f}")
# Add text A and B to the image
ax_image_scurve.text(50, 50, "A", color="white", fontsize=20, ha="center", va="center")
ax_image_scurve.text(150, 50, "B", color="black", fontsize=20, ha="center", va="center")


# ======================================================================================
# Plot the S-curve
# ======================================================================================
s_curve_in = np.linspace(0, 1, 100)
s_curve_out = s_curve(s_curve_in)
ax_s_curve.plot(s_curve_in, s_curve_out)
ax_s_curve.set_aspect(0.5)
ax_s_curve.set_title("S-curve")
ax_s_curve.set_xlabel("Input intensity")
ax_s_curve.set_ylabel("Output intensity")

plt.tight_layout()

# ======================================================================================
# Save the figure
# ======================================================================================
output_dir = Path("../content/assets")
output_dir.mkdir(exist_ok=True, parents=True)

plt.savefig(output_dir/"gcnr-cnr-comparison.png", dpi=300, bbox_inches="tight")

# Visualize overlapping regions used by gCNR

In [ ]:
# Create x-axis values for the probability density functions
x = np.linspace(0, 1, 1000)

# Compute the minimum of the two probability density functions
min_pa_pb = np.min(np.stack([p_a(x), p_b(x)], axis=0), axis=0)


# Get the default color cycle
color_cycle = plt.rcParams['axes.prop_cycle'].by_key()['color']

print('0EA7FF')
print(color_cycle[1])

fig, ax = plt.subplots(1, 1, figsize=(7, 2.5))
ax.plot(x, p_a(x), label="$p_A(x)$", linewidth=1.5)
ax.plot(x, p_b(x), label="$p_B(x)$", linewidth=1.5)

ax.set_xlabel("Pixel intensity")
ax.set_ylabel("Probability density")
ax.fill_between(x, min_pa_pb, hatch="////", edgecolor=color_cycle[2], facecolor="k",alpha=1.0, label="overlap $\int \min(p_A(x), p_B(x))dx$")
# Add a legend outside the plot
leg = ax.legend(loc='center left', bbox_to_anchor=(1, 0.5))

ylims = ax.get_ylim()

plt.tight_layout()

# ======================================================================================
# Save the figure
# ======================================================================================
plt.savefig(output_dir/"gcnr-overlap.png", dpi=300, bbox_inches="tight")


In [ ]:
# Compute the normalization factor for the probability density functions to ensure that
# the area under the s-curve transformed probability density functions is equal to 1
normalization_factor = np.sum(s_curve(p_a(x)))/np.sum(p_a(x))

# Compute the s-curve transformed probability density functions
p_a_transformed = s_curve(p_a(x))/normalization_factor
p_b_transformed = s_curve(p_b(x))/normalization_factor

# Compute the minimum of the two transformed probability density functions
min_pa_pb = np.min(np.stack([p_a_transformed, p_b_transformed], axis=0), axis=0)

# ======================================================================================
# Create a plot of the s-curve transformed probability density functions
# ======================================================================================
fig, ax = plt.subplots(1, 1, figsize=(7, 2.5))
ax.plot(x, p_a_transformed, label="$p_A(x)$", linewidth=1.5)
ax.plot(x, p_b_transformed, label="$p_B(x)$", linewidth=1.5)


ax.set_xlabel("Pixel intensity")
ax.set_ylabel("Probability density")
ax.fill_between(x, min_pa_pb, hatch="////", edgecolor=color_cycle[2], facecolor="k",alpha=1.0, label="overlap $\int \min(p_A(x), p_B(x))dx$")
# Add a legend outside the plot
leg = ax.legend(loc='center left', bbox_to_anchor=(1, 0.5))
ax.set_ylim(ylims)

plt.tight_layout()

# ======================================================================================
# Save the figure
# ======================================================================================
plt.savefig(output_dir/"gcnr-scurve-overlap.png", dpi=300, bbox_inches="tight")
